In [172]:
import pandas as pd
import pingouin as pg
import lightgbm as lgb
import numpy as np
import shap
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.metrics import classification_report, roc_auc_score

plt.rcParams["font.family"] = "Malgun Gothic"  

df = pd.read_csv("preprocessing_filtered.csv")
df

,Unnamed: 0,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,price,estimated_occupancy_l365d,estimated_revenue_l365d,availability_60,availability_365,...,neighborhood_overview_exists,host_about_length_group,host_is_superhost,host_has_profile_pic,host_identity_verified,host_response_time_score,host_response_rate_score,host_acceptance_rate_score,host_location_ny,host_location_boolean
0,0,Prospect Heights,Brooklyn,40.673760,-73.966110,200.0,0,0.0,57,362,...,1,long,0,1,1,4.0,4,4,1,1
1,1,East Harlem,Manhattan,40.792454,-73.940742,82.0,0,0.0,0,204,...,1,empty,0,1,1,4.0,4,4,1,1
2,2,Boerum Hill,Brooklyn,40.684420,-73.980680,765.0,0,0.0,49,326,...,1,long,0,1,1,3.0,4,2,1,1
3,3,Harlem,Manhattan,40.818058,-73.946671,139.0,255,35445.0,18,25,...,1,long,1,1,1,4.0,4,4,1,1
4,4,Williamsburg,Brooklyn,40.710651,-73.950874,130.0,154,20020.0,22,38,...,1,long,1,1,1,4.0,4,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22303,22303,Harlem,Manhattan,40.813794,-73.952399,72.0,0,0.0,53,358,...,0,empty,0,1,1,4.0,4,4,1,1
22304,22304,Bedford-Stuyvesant,Brooklyn,40.685500,-73.919510,58.0,0,0.0,58,363,...,0,long,1,1,1,4.0,4,4,1,1
22305,22305,Kips Bay,Manhattan,40.745290,-73.979380,299.0,0,0.0,39,99,...,0,empty,0,1,1,4.0,4,4,0,1
22306,22306,Theater District,Manhattan,40.757133,-73.983124,200.0,0,0.0,60,365,...,0,long,0,1,1,4.0,4,4,1,1


In [173]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22308 entries, 0 to 22307
Data columns (total 45 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    22308 non-null  int64  
 1   neighbourhood_cleansed        22308 non-null  object 
 2   neighbourhood_group_cleansed  22308 non-null  object 
 3   latitude                      22308 non-null  float64
 4   longitude                     22308 non-null  float64
 5   price                         22308 non-null  float64
 6   estimated_occupancy_l365d     22308 non-null  int64  
 7   estimated_revenue_l365d       22308 non-null  float64
 8   availability_60               22308 non-null  int64  
 9   availability_365              22308 non-null  int64  
 10  instant_bookable              22308 non-null  int64  
 11  is_long_term                  22308 non-null  int64  
 12  is_activate                   22308 non-null  int64  
 13  a

In [174]:
df.columns

Index(['Unnamed: 0', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed',
       'latitude', 'longitude', 'price', 'estimated_occupancy_l365d',
       'estimated_revenue_l365d', 'availability_60', 'availability_365',
       'instant_bookable', 'is_long_term', 'is_activate', 'amenities',
       'amenities_cnt', 'room_type', 'structure_type', 'structure_category',
       'accommodates', 'bath_score_mul', 'beds', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'id', 'host_id', 'name_length_group',
       'description_length_group', 'neighborhood_overview_exists',
       'host_about_length_group', 'host_is_superhost', 'host_has_profile_pic',
       'host_identity_verified', 'host_response_time_score',
       'host_response_rate_score', 'host_acce

In [175]:
# price 정규성 검정
price_data = df['price'].dropna()

stat, p = stats.shapiro(price_data)

print(f"Shapiro-Wilk 통계량: {stat:.4f}") #실제 데이터가 정규분포에 얼마나 가까운지를 수치화한 값 (0~1 사이)
print(f"p-value: {p:.4f}")
if p > 0.05:
    print("정규성을 가정할 수 없다")
else:
    print("정규성을 가정할 수 없다")


Shapiro-Wilk 통계량: 0.2379
p-value: 0.0000
정규성을 가정할 수 없다


c:\Users\HY\AppData\Local\Programs\Python\Python313\Lib\site-packages\scipy\stats\_axis_nan_policy.py:579: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 22308.
  res = hypotest_fun_out(*samples, **kwds)


In [176]:
#로그변환 

log_price = np.log1p(df['price'].dropna())  # log(1 + price)로 0도 처리 가능
stat, p = stats.shapiro(log_price)

print(f"Shapiro-Wilk 통계량: {stat:.4f}")
print(f"p-value: {p:.4f}")

if p > 0.05:
    print("로그 변환 후 정규성을 가정할 수 있다")
else:
    print("로그 변환 후에도 정규성을 가정할 수 없다")

Shapiro-Wilk 통계량: 0.9811
p-value: 0.0000
로그 변환 후에도 정규성을 가정할 수 없다


In [177]:
from scipy.stats import mannwhitneyu

group1 = df[df['host_is_superhost'] == 1]['price'].dropna()
group0 = df[df['host_is_superhost'] == 0]['price'].dropna()

stat, p = mannwhitneyu(group1, group0, alternative='two-sided')
print(f"Mann-Whitney U 통계량: {stat:.4f}, p-value: {p:.4f}")

# 슈퍼호스트인지 아닌지 두그룹의 가격 분포에 차이가 있다

Mann-Whitney U 통계량: 45228149.5000, p-value: 0.0000


In [178]:
import pandas as pd
from scipy.stats import shapiro, ttest_ind, mannwhitneyu, chi2_contingency
import pingouin as pg   # 카이-제곱용

TARGET = 'host_is_superhost'

# 수치형 변수 리스트 (위도·경도·식별자 제외)
raw_num = [c for c in df.select_dtypes(include=['int64','float64']).columns
           if c not in ['latitude','longitude','host_id','id','host_is_superhost','Unnamed: 0']]

# 이진 수치형(0/1)만 골라내기
binary_num = [c for c in raw_num if df[c].dropna().isin([0,1]).all()]
continuous_num = [c for c in raw_num if c not in binary_num]

# 범주형 변수
cat_cols = df.select_dtypes(include=['object','category']).columns

results = []

# 연속형: 정규성 → t vs Mann-Whitney
def check_normality(series):
    return shapiro(series.dropna())[1] >= 0.05

for col in continuous_num:
    super = df[df[TARGET]==1][col].dropna()
    non   = df[df[TARGET]==0][col].dropna()
    
    if check_normality(super) and check_normality(non):
        stat, p = ttest_ind(super, non, equal_var=False)
        test = 't-test'
    else:
        stat, p = mannwhitneyu(super, non, alternative='two-sided')
        test = 'Mann-Whitney U'
    
    results.append({'variable':col, 'test':test, 'p':round(p,4)})

# 이진 수치형 & 범주형 → 카이제곱
for col in binary_num + cat_cols.tolist():
    ct = pd.crosstab(df[col], df[TARGET])
    chi2, p, _, _ = chi2_contingency(ct)
    results.append({'variable':col, 'test':'chi2', 'p':round(p,4)})

# 결과 정리
stat_df = pd.DataFrame(results).sort_values('p')
stat_df

c:\Users\HY\AppData\Local\Programs\Python\Python313\Lib\site-packages\scipy\stats\_axis_nan_policy.py:579: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 6135.
  res = hypotest_fun_out(*samples, **kwds)


,variable,test,p
0,price,Mann-Whitney U,0.0000
1,estimated_occupancy_l365d,Mann-Whitney U,0.0000
2,estimated_revenue_l365d,Mann-Whitney U,0.0000
3,availability_60,Mann-Whitney U,0.0000
4,availability_365,Mann-Whitney U,0.0000
5,amenities_cnt,Mann-Whitney U,0.0000
9,number_of_reviews,Mann-Whitney U,0.0000
8,beds,Mann-Whitney U,0.0000
12,review_scores_rating,Mann-Whitney U,0.0000
13,review_scores_accuracy,Mann-Whitney U,0.0000


슈퍼호스트인지 아닌지는 accommodates(수용인원)과 host_identity_verified(호스트신원확인유무)와 관계없다 
귀무가설 기각하지 못함

In [179]:
superhost_median = df[df['host_is_superhost'] == 1]['price'].median()
non_superhost_median = df[df['host_is_superhost'] == 0]['price'].median()

print(f"슈퍼호스트 중앙값: {superhost_median}")
print(f"일반 호스트 중앙값: {non_superhost_median}")

# 슈퍼호스트 중앙값이 더 낮다. 합리적인 가격이 슈퍼호스트가 되기에 유리하다값이 나옴 

슈퍼호스트 중앙값: 129.0
일반 호스트 중앙값: 147.0


In [180]:
# 수치형연속형번수만 
continuous_cols = [
    c for c in df.select_dtypes(include=['int64', 'float64']).columns
    if c not in [
        'host_is_superhost', # 종속변수
        'latitude', 'longitude', 'host_id', 'id', 'Unnamed: 0',
        # 이진 0/1 변수들 추가
        'host_identity_verified', 'host_location_boolean', 'host_location_ny',
        'neighborhood_overview_exists', 'is_long_term', 'instant_bookable',
        'is_activate', 'host_has_profile_pic','accommodates'
    ]
]

# 중앙값 테이블
median_table = pd.DataFrame({
    'variable': continuous_cols,
    'superhost_median': [df[df['host_is_superhost'] == 1][col].median() for col in continuous_cols],
    'non_superhost_median': [df[df['host_is_superhost'] == 0][col].median() for col in continuous_cols]
})

# 3) 확인
median_table.sort_values('superhost_median', ascending=False)

,variable,superhost_median,non_superhost_median
2,estimated_revenue_l365d,16830.00,0.00
4,availability_365,233.00,270.00
1,estimated_occupancy_l365d,166.00,0.00
0,price,129.00,147.00
5,amenities_cnt,37.00,29.00
3,availability_60,29.00,53.00
8,number_of_reviews,22.00,2.00
15,review_scores_communication,4.95,4.81
14,review_scores_checkin,4.94,4.83
12,review_scores_accuracy,4.90,4.75


In [181]:
# 수치형연속형번수만 
continuous_cols = [
    c for c in df.select_dtypes(include=['int64', 'float64']).columns
    if c not in [
        'host_is_superhost', # 종속변수
        'latitude', 'longitude', 'host_id', 'id', 'Unnamed: 0',
        # 이진 0/1 변수들 추가
        'host_identity_verified', 'host_location_boolean', 'host_location_ny',
        'neighborhood_overview_exists', 'is_long_term', 'instant_bookable',
        'is_activate', 'host_has_profile_pic','accommodates'
    ]
]

# 평균값 테이블 
avg_table = pd.DataFrame({
    'variable': continuous_cols,
    'superhost_avg': [df[df['host_is_superhost'] == 1][col].mean().round(2) for col in continuous_cols],
    'non_superhost_avg': [df[df['host_is_superhost'] == 0][col].mean().round(2) for col in continuous_cols]
})

# 3) 확인
avg_table.sort_values('superhost_avg', ascending=False)

,variable,superhost_avg,non_superhost_avg
2,estimated_revenue_l365d,23350.91,7558.93
4,availability_365,219.54,247.38
0,price,178.78,227.13
1,estimated_occupancy_l365d,141.79,44.92
8,number_of_reviews,55.23,26.43
5,amenities_cnt,36.41,28.51
3,availability_60,29.06,38.86
9,number_of_reviews_ltm,11.69,4.03
15,review_scores_communication,4.91,4.78
14,review_scores_checkin,4.90,4.80


In [182]:
# 평균 + 중앙값 테이블 합치기
merged_table = pd.merge(
    avg_table,
    median_table,
    on='variable'
)

# 차이 컬럼 추가
merged_table['mean_diff'] = (merged_table['superhost_avg'] - merged_table['non_superhost_avg']).round(2)
merged_table['median_diff'] = (merged_table['superhost_median'] - merged_table['non_superhost_median']).round(2)

# 차이 기준 정렬 (예: 평균 차이가 큰 순서)
merged_table.sort_values('mean_diff', ascending=False)

,variable,superhost_avg,non_superhost_avg,superhost_median,non_superhost_median,mean_diff,median_diff
2,estimated_revenue_l365d,23350.91,7558.93,16830.00,0.00,15791.98,16830.00
1,estimated_occupancy_l365d,141.79,44.92,166.00,0.00,96.87,166.00
8,number_of_reviews,55.23,26.43,22.00,2.00,28.80,20.00
5,amenities_cnt,36.41,28.51,37.00,29.00,7.90,8.00
9,number_of_reviews_ltm,11.69,4.03,3.00,0.00,7.66,3.00
20,host_acceptance_rate_score,3.73,3.43,4.00,4.00,0.30,0.00
10,number_of_reviews_l30d,0.53,0.24,0.00,0.00,0.29,0.00
19,host_response_rate_score,3.99,3.80,4.00,4.00,0.19,0.00
11,review_scores_rating,4.84,4.68,4.88,4.72,0.16,0.16
17,review_scores_value,4.73,4.57,4.78,4.61,0.16,0.17


In [183]:
# 이진수치형 데이터 변수
bin_vars = ['host_location_boolean', 'host_location_ny',
        'neighborhood_overview_exists', 'is_long_term', 'instant_bookable',
        'is_activate', 'host_has_profile_pic'] 
# 비교 테이블 생성
bin_table = pd.DataFrame({
    'variable': bin_vars,
    'superhost_ratio(%)': [df[df['host_is_superhost'] == 1][col].mean().round(2) * 100 for col in bin_vars],
    'non_superhost_ratio(%)': [df[df['host_is_superhost'] == 0][col].mean().round(2) * 100 for col in bin_vars]
})

# 차이 컬럼 추가
bin_table['ratio_diff(%)'] = (bin_table['superhost_ratio(%)'] - bin_table['non_superhost_ratio(%)']).round(2)

# 정렬 (차이 큰 순서)
bin_table = bin_table.sort_values('ratio_diff(%)', ascending=False)

# 확인
bin_table

# is_activate 활성숙소가 슈퍼호스트는 87% 비슈퍼호스트는 62%
# 브루클린에 사는 호스트중 비 슈퍼호스트는 74% 슈퍼호스트는 80%
# 숙소주변 소개글이 있는 걸로 비율계산 해서 슈퍼호스트는 61% 비 슈퍼 호스트는 53%

,variable,superhost_ratio(%),non_superhost_ratio(%),ratio_diff(%)
5,is_activate,87.0,62.0,25.0
2,neighborhood_overview_exists,61.0,53.0,8.0
1,host_location_ny,74.0,66.0,8.0
0,host_location_boolean,81.0,75.0,6.0
6,host_has_profile_pic,99.0,96.0,3.0
4,instant_bookable,20.0,28.0,-8.0
3,is_long_term,71.0,84.0,-13.0


범주형 데이터 - 
'neighbourhood_cleansed','neighbourhood_group_cleansed','structure_type','structure_category','name_length_group','description_length_group','host_about_length_group' 

In [184]:
#범주형변수 룸타입 
cat_var_room = 'room_type'
ct = pd.crosstab(df[cat_var], df['host_is_superhost'], normalize='columns') * 100
ct.round(1)
# 비슈퍼호스트일때와 슈퍼호스트일때 룸타입별 비율 

host_is_superhost,0,1
room_type,,
Entire home/apt,56.1,58.7
Hotel room,2.3,0.0
Private room,41.3,40.8
Shared room,0.4,0.5


In [185]:
cat_var_name = 'name_length_group'
ct = pd.crosstab(df[cat_var_name], df['host_is_superhost'], normalize='columns') * 100
ct.round(1)

host_is_superhost,0,1
name_length_group,,
long,47.8,53.2
short_or_med,52.2,46.8


In [186]:
cat_var_description = 'description_length_group'
ct = pd.crosstab(df[cat_var_description], df['host_is_superhost'], normalize='columns') * 100
ct.round(1)

host_is_superhost,0,1
description_length_group,,
empty,2.1,1.1
long,54.3,65.1
short_or_avg,43.6,33.8


In [187]:
cat_var_hostabout = 'host_about_length_group'
ct = pd.crosstab(df[cat_var_hostabout], df['host_is_superhost'], normalize='columns') * 100
ct.round(1)

host_is_superhost,0,1
host_about_length_group,,
empty,41.7,35.5
long,48.3,54.5
short_or_med,10.1,10.0


In [188]:
cat_var_structure = 'structure_type'
ct = pd.crosstab(df[cat_var_structure], df['host_is_superhost'], normalize='columns') * 100
ct.round(1)

host_is_superhost,0,1
structure_type,,
aparthotel,0.2,0.3
barn,0.0,0.0
bed and breakfast,0.3,0.3
boat,0.0,0.0
boutique hotel,1.2,0.2
bungalow,0.1,0.1
camper/rv,0.1,0.0
casa particular,0.2,0.2
condo,4.3,5.6


In [189]:
cat_var_neighbourhood = 'neighbourhood_cleansed'
ct = pd.crosstab(df[cat_var_neighbourhood], df['host_is_superhost'], normalize='columns') * 100
ct.round(1)

host_is_superhost,0,1
neighbourhood_cleansed,,
Allerton,0.1,0.3
Arden Heights,0.0,0.0
Arrochar,0.0,0.0
Arverne,0.3,0.4
Astoria,1.2,1.7
...,...,...
Windsor Terrace,0.2,0.3
Woodhaven,0.3,0.4
Woodlawn,0.0,0.1


In [190]:
cat_var_group = 'neighbourhood_group_cleansed'
ct = pd.crosstab(df[cat_var_group], df['host_is_superhost'], normalize='columns') * 100
ct.round(1)

host_is_superhost,0,1
neighbourhood_group_cleansed,,
Bronx,4.1,4.0
Brooklyn,31.3,38.9
Manhattan,48.8,37.6
Queens,14.4,17.8
Staten Island,1.3,1.8


In [191]:
# 제외할 칼럼
exclude_cols = ['host_id', 'id', 'latitude', 'longitude', 'host_is_superhost']

# 이진 수치형 변수 'host_identity_verified'귀무가설 기각 못해서 제외함
bin_vars = ['host_location_boolean', 'host_location_ny',
        'neighborhood_overview_exists', 'is_long_term', 'instant_bookable',
        'is_activate', 'host_has_profile_pic'] 

# 전체 수치형 변수에서 이진 변수 제외
numeric_vars = [col for col in df.select_dtypes(include=['int64', 'float64']).columns
                if col not in exclude_cols + bin_vars]

# 범주형 변수
categorical_vars = df.select_dtypes(include='object').columns.tolist()

# X, y 설정
X = df[numeric_vars + bin_vars + categorical_vars + [TARGET]].copy()
y = X.pop(TARGET).astype(int) 

# 인코딩위해서 범주형과 숫자형(이진,숫자)으로 나누기 
cat_cols = [c for c in X.columns if X[c].dtype == 'object']   # 문자만
num_cols = [c for c in X.columns if X[c].dtype != 'object'] 

# 학습,테스트 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
from sklearn.preprocessing import StandardScaler

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# 범주형만 원핫인코딩
X_cat_train = encoder.fit_transform(X_train[cat_cols])
X_cat_test  = encoder.transform(X_test[cat_cols])

# 숫자는 그대로
X_num_train = X_train[num_cols].values
X_num_test  = X_test[num_cols].values

# 스케일링 
scaler = StandardScaler()

X_num_train = scaler.fit_transform(X_num_train)
X_num_test  = scaler.transform(X_num_test)

# 두 개 합치기
import numpy as np
X_train_all = np.hstack([X_num_train, X_cat_train])
X_test_all  = np.hstack([X_num_test,  X_cat_test])

# 사용할 모델 
models = {'Logistic':   LogisticRegression(max_iter=3000, class_weight='balanced'),
    'RandomForest': RandomForestClassifier(n_estimators=300,
                                           random_state=42,
                                           class_weight='balanced'),
    'LightGBM':   lgb.LGBMClassifier(n_estimators=300,
                                     learning_rate=0.05,
                                     random_state=42,
                                     class_weight='balanced')}

for name, model in models.items():
    model.fit(X_train_all, y_train)
    
    pred = model.predict(X_test_all)
    prob = model.predict_proba(X_test_all)[:, 1]  # AUC 계산용 1에 대한 확률만
    
    auc = roc_auc_score(y_test, prob)
    
    print(f"\n=== {name} ===")
    print(classification_report(y_test, pred))
    print("AUC:", round(auc, 4))

c:\Users\HY\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



=== Logistic ===
              precision    recall  f1-score   support

           0       0.87      0.77      0.82      3235
           1       0.54      0.69      0.60      1227

    accuracy                           0.75      4462
   macro avg       0.70      0.73      0.71      4462
weighted avg       0.78      0.75      0.76      4462

AUC: 0.7865

=== RandomForest ===
              precision    recall  f1-score   support

           0       0.87      0.95      0.91      3235
           1       0.82      0.63      0.71      1227

    accuracy                           0.86      4462
   macro avg       0.84      0.79      0.81      4462
weighted avg       0.86      0.86      0.85      4462

AUC: 0.9166


LightGBMError: Do not support special JSON characters in feature name.

In [196]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

# 제외할 칼럼 및 이진 변수 지정
exclude_cols = ['host_id', 'id', 'latitude', 'longitude', 'host_is_superhost']
bin_vars = ['host_location_boolean', 'host_location_ny',
            'neighborhood_overview_exists', 'is_long_term', 'instant_bookable',
            'is_activate', 'host_has_profile_pic']

# 수치형 / 범주형 변수 지정
numeric_vars = [col for col in df.select_dtypes(include=['int64', 'float64']).columns
                if col not in exclude_cols + bin_vars]
categorical_vars = df.select_dtypes(include='object').columns.tolist()

# X, y 분리
TARGET = 'host_is_superhost'
X = df[numeric_vars + bin_vars + categorical_vars + [TARGET]].copy()
y = X.pop(TARGET).astype(int)

# 학습/테스트 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# -----------------------
# 1. ColumnTransformer 설정
# -----------------------

# 숫자형: 스케일링 / 범주형: 원핫인코딩 / 이진: 그대로 (패스)
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_vars),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_vars),
    ('bin', 'passthrough', bin_vars)], remainder='drop', verbose_feature_names_out=False).set_output(transform='pandas')  

# -----------------------
# 2. 사용할 모델들 정의
# -----------------------
models = {
    'Logistic': LogisticRegression(max_iter=1000, class_weight='balanced'),
    'RandomForest': RandomForestClassifier(n_estimators=300, random_state=42, class_weight='balanced'),
    'LightGBM': lgb.LGBMClassifier(n_estimators=300, learning_rate=0.05, random_state=42, class_weight='balanced')
}

# -----------------------
# 3. 모델 학습 및 평가
# -----------------------
for name, clf in models.items():
    pipe = Pipeline(steps=[
        ('preprocess', preprocessor),
        ('classifier', clf)
    ])

    # 학습
    pipe.fit(X_train, y_train)

    # 예측
    y_pred = pipe.predict(X_test)
    y_prob = pipe.predict_proba(X_test)[:, 1]

    # 평가
    print(f"\n=== {name} ===")
    print(classification_report(y_test, y_pred))
    print("AUC:", round(roc_auc_score(y_test, y_prob), 4))


=== Logistic ===
              precision    recall  f1-score   support

           0       0.89      0.81      0.85      3235
           1       0.60      0.74      0.66      1227

    accuracy                           0.79      4462
   macro avg       0.75      0.78      0.76      4462
weighted avg       0.81      0.79      0.80      4462

AUC: 0.8582

=== RandomForest ===
              precision    recall  f1-score   support

           0       0.87      0.95      0.91      3235
           1       0.82      0.63      0.71      1227

    accuracy                           0.86      4462
   macro avg       0.84      0.79      0.81      4462
weighted avg       0.86      0.86      0.85      4462

AUC: 0.9165


LightGBMError: Do not support special JSON characters in feature name.